# Information

The lambda will be zipped and uplodaded to an S3, where it will trigger CloudFormation to create stack. The stack will create an API Gateway and the necessary role. The S3 bucket and the SNS topic won't be created by CloudFormation though so this file includes the CLI commands to do it. Alternatively, use existing ones and skip the creation cells.

The CLI commands for tags are commented out. They are mainly for information purposes. Nevertheless, remember that tagging resources is a best practice.

# Function information

By default, the function name will be the folder name of the .NET project

In [ ]:
$functionName = Split-Path -Path $pwd -Leaf
Write-Host "Function name: $functionName"

$functionNameLower = ($functionName).ToLower()
Write-Host "Function name lowercase: $functionNameLower"

$tagKeyValue = "Key=lambda,Value='$functionNameLower'"
Write-Host "Tag key value: $tagKeyValue"

$tagset = "TagSet=[{$tagKeyValue}]"
Write-Host "Tag set: $tagset"

# Prerequisites

## S3 Bucket

In [ ]:
$s3Bucket = "play-with-lambdas"
Write-Host "S3 Bucket: $s3Bucket"
$s3BucketUri = "s3://" + $s3Bucket
Write-Host "S3 Bucket URI: $s3BucketUri"

If it doesn't exist, create it

In [ ]:
aws s3 mb $s3BucketUri
#aws s3api put-bucket-tagging --bucket $s3Bucket --tagging $tagset

## SNS Topic

In [ ]:
$snsTopicName = $functionName
Write-Host "SNS Topic: $snsTopicName" 
$snsTopicArn = "arn:aws:sns:us-west-1:128163920149:LambdaMinimalApiPublishesOnSNS"

If it doesn't exist, create it

In [ ]:
$out = aws sns create-topic --name $snsTopicName
$snsTopicArn = ($out | ConvertFrom-Json).TopicArn
Write-Host "SNS Topic ARN: $snsTopicArn"
#aws sns tag-resource --resource $snsTopicArn --tags $tagKeyValue

# Deploy

Deploy the function

In [ ]:
dotnet lambda deploy-serverless --stack-name $functionName --s3-bucket $s3Bucket
#aws lambda tag-resource --resource $functionArn --tags $tagKeyValue

If "Run All", just sleep because the Polyglot notebooks just sucks.

In [ ]:
Write-Host "Sleeping for 5s..."
Start-Sleep -Seconds 5
Write-Host "Back to business!"

Get the deployment details

In [ ]:
$deployedFunctionName = aws lambda list-functions --query "Functions[?starts_with(FunctionName, '$functionName')].FunctionName" --output text
Write-Host "Deployed function name: $deployedFunctionName"

$function = aws lambda get-function --function-name $deployedFunctionName
$functionJson = $function | ConvertFrom-Json
Write-Host "Function ARN: $functionArn = $functionJson.Configuration.FunctionArn"

$roleArn = $functionJson.Configuration.Role
$roleName = ($roleArn).split("/")[1]
Write-Host "Role nane: $roleName"

$cloudfrontStack = aws cloudformation describe-stacks --stack-name $functionName
$cloudfrontStackJson = $cloudfrontStack | ConvertFrom-Json
$apiGateWayUrl = $cloudfrontStackJson.Stacks[0].Outputs[0].OutputValue
Write-Host "Api Gateway URL: $apiGateWayUrl"

$apiGateWayId = [regex]::match($apiGateWayUrl,'https://([A-Za-z0-9]+)').Groups[1].Value
Write-Host "API Gateway ID: $apiGateWayId"

Enable tracing

In [ ]:
Write-Host "Activating tracing for lambda..."
aws lambda update-function-configuration --function-name $deployedFunctionName --tracing-config Mode=Active

Write-Host "Activating tracing for API Gateway..."
aws apigateway update-stage --rest-api-id $apiGateWayId --stage-name Prod --patch-operations op=replace,path=/tracingEnabled,value=true

Allow the lambda to publish in SNS

In [ ]:
Write-Host "Allowing lambda to publish to SNS..."
aws iam attach-role-policy --policy-arn arn:aws:iam::aws:policy/AmazonSNSFullAccess --role-name $roleName

Add the SNS Topic ARN as env variable of the function

In [ ]:
Write-Host "Adding sns topic arn to lambda env variables..."
aws lambda update-function-configuration --function-name $deployedFunctionName --environment "Variables={AWS_SNS_TOPICARN=$snsTopicArn}"

# <p style="text-align: center;">🎉 Deployed!</p>

# Test it

If "Run All", just sleep because the Polyglot notebooks just sucks.

In [ ]:
Write-Host "Sleeping for 5s..."
Start-Sleep -Seconds 5
Write-Host "Back to business!"

In [ ]:
$payload = '{
    "name": "Joan Comas",
    "address": "Awesome street 7, Nicetown, Amazingland"
}'
Invoke-WebRequest -Method Post -Uri ($apiGateWayUrl + "users") -ContentType "application/json" -Body $payload

# Clean up

In [ ]:
return
aws iam detach-role-policy --role-name $roleName --policy-arn arn:aws:iam::aws:policy/AmazonSNSFullAccess
aws cloudformation delete-stack --stack-name $functionName
aws sns delete-topic --topic-arn $snsTopicArn
aws s3 rb --force $s3BucketUri